<a href="https://colab.research.google.com/github/creative-h/agentQ_Travel_Planner/blob/main/travel_booking_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Travel Package Booking Portal

This notebook demonstrates how to use the travel package booking portal to search for and book travel packages that combine flights and hotels.

## 1. Setup and Configuration

First, let's install the required dependencies:

In [ ]:
!git clone https://github.com/creative-h/agentQ_Travel_Planner.git

In [ ]:
!unzip agentQ_Travel_Planner/travelAgentQ.zip

In [1]:
cd /content/travelAgentQ/travel_booking_portal


/content/travelAgentQ/travel_booking_portal


In [ ]:
!pip install "httpx<0.28"
!pip install -r requirements.txt

Now, let's set up our API credentials. You'll need to obtain API keys for:
- Amadeus (for flight and hotel data): https://developers.amadeus.com/
- Groq (for AI-powered recommendations): https://console.groq.com/

In [2]:
import os
import sys
import logging

# Add parent directory to path to import modules
sys.path.append('..')

# Set up API credentials
os.environ['AMADEUS_API_KEY'] = "bsknoiobYmGP9Ur47t6bGwFDUMihUtnH"  # Replace with your actual API key
os.environ['AMADEUS_API_SECRET'] = "4YN13VQCc0ti4X8I"  # Replace with your actual API secret
os.environ['GROQ_API_KEY'] = "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"  # Replace with your actual API key
os.environ['GROQ_MODEL'] = "llama-3.3-70b-versatile"  # You can change this to a different model if needed

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('notebook')

## 2. Initialize APIs and Agent

Now, let's initialize the APIs and agent for our travel booking portal:

In [3]:
from utils import Config, TravelAgent, DataProcessor
from api import FlightAPI, HotelAPI, PackageAPI

# Load configuration
config = Config()
if not config.validate():
    logger.error("Invalid configuration. Please check your API credentials.")
    raise ValueError("Invalid configuration")

# Initialize APIs
amadeus_credentials = config.get_amadeus_credentials()
flight_api = FlightAPI(amadeus_credentials['api_key'], amadeus_credentials['api_secret'])
hotel_api = HotelAPI(amadeus_credentials['api_key'], amadeus_credentials['api_secret'])
package_api = PackageAPI(flight_api, hotel_api)

# Initialize agent
groq_credentials = config.get_groq_credentials()
travel_agent = TravelAgent(groq_credentials['api_key'], groq_credentials['model'])

# Initialize data processor
data_processor = DataProcessor()

print("APIs and agent initialized successfully!")

APIs and agent initialized successfully!


## 3. Natural Language Query Processing

Let's use the travel agent to process a natural language query and extract travel criteria:

In [4]:
# Example natural language query
query = "I want to travel from New York to Paris for a week in July with my family (2 adults, 1 child)"

# Extract travel criteria
criteria = travel_agent.extract_travel_criteria(query)

# Display extracted criteria
import json
print(json.dumps(criteria, indent=2))

{
  "origin_location": "New York",
  "destinations": [
    "Paris"
  ],
  "departure_date": "July",
  "return_date": "July",
  "adults": 2,
  "children": 1,
  "infants": 0,
  "travel_class": null,
  "budget": null,
  "preferences": null,
  "multi_destination": false
}


## 4. Search for Flights

Now, let's search for flights based on the extracted criteria:

In [5]:
# Set up search parameters
origin = criteria.get('origin_location', 'NYC')  # Default to NYC if not extracted
destination = criteria.get('destinations', 'PAR')  # Default to Paris if not extracted
if isinstance(destination, list):
    destination = destination[0]  # Take the first destination if multiple are extracted

# Set dates (default to next month if not extracted)
from datetime import datetime, timedelta
default_departure = (datetime.now() + timedelta(days=30)).strftime('%Y-%m-%d')
default_return = (datetime.now() + timedelta(days=37)).strftime('%Y-%m-%d')
departure_date = criteria.get('departure_date', default_departure)
return_date = criteria.get('return_date', default_return)

# Set passenger counts
adults = criteria.get('adults', 2)
children = criteria.get('children', 1)

# Set travel class
travel_class = criteria.get('travel_class', 'ECONOMY')

# Search for flights
print(f"Searching for flights from {origin} to {destination} on {departure_date} returning {return_date}...")
flight_offers = flight_api.search_flights(
    origin_location_code=origin,
    destination_location_code=destination,
    departure_date=departure_date,
    return_date=return_date,
    adults=adults,
    children=children,
    travel_class=travel_class,
    currency_code="USD",
    max_results=5
)

# Display flight offers
print(f"Found {len(flight_offers)} flight offers")
if flight_offers:
    # Parse flight data into a DataFrame
    flights_df = data_processor.parse_flight_data(flight_offers)
    flights_df.head()

Searching for flights from New York to Paris on July returning July...


ERROR:flight_api:Error searching flights: [400]




ERROR:flight_api:Error code: ClientError
ERROR:flight_api:Error message: <bound method ResponseError.description of ClientError('[400]\n\n\n\n')>
ERROR:flight_api:Full error response: {"errors":[{"status":400,"code":477,"title":"INVALID FORMAT","detail":"departureDate format is YYYY-MM-DD","source":{"pointer":"departureDate","example":"2030-12-31"}},{"status":400,"code":477,"title":"INVALID FORMAT","detail":"destinationLocationCode must be a 3-letter code","source":{"pointer":"destinationLocationCode","example":"LON"}},{"status":400,"code":477,"title":"INVALID FORMAT","detail":"originLocationCode must be a 3-letter code","source":{"pointer":"originLocationCode","example":"PAR"}},{"status":400,"code":477,"title":"INVALID FORMAT","detail":"returnDate format is YYYY-MM-DD","source":{"pointer":"returnDate","example":"2030-12-31"}}]}


Found 0 flight offers


In [9]:
from datetime import datetime, timedelta
import re

# Helper: Convert month name + duration to real dates
def infer_dates(departure_text: str = None, trip_duration: int = 7):
    try:
        if departure_text and re.match(r'^[A-Za-z]+$', departure_text.strip()):
            month_number = datetime.strptime(departure_text.strip(), "%B").month
            today = datetime.today()
            year = today.year if today.month <= month_number else today.year + 1
            departure = datetime(year, month_number, 10)  # default to 10th
            return departure.strftime('%Y-%m-%d'), (departure + timedelta(days=trip_duration)).strftime('%Y-%m-%d')
    except Exception:
        pass
    # Fallback to next month
    default_departure = datetime.now() + timedelta(days=30)
    default_return = default_departure + timedelta(days=trip_duration)
    return default_departure.strftime('%Y-%m-%d'), default_return.strftime('%Y-%m-%d')


# Set up search parameters
origin_city = criteria.get('origin_location', 'NYC')
destination_city = criteria.get('destinations', ['PAR'])
if isinstance(destination_city, list):
    destination_city = destination_city[0]

# Map to IATA codes (add more as needed)
iata_map = {
    'New York': 'NYC',
    'Paris': 'PAR',
    'London': 'LON',
    'Tokyo': 'TYO'
}
origin = iata_map.get(origin_city, origin_city[:3].upper())
destination = iata_map.get(destination_city, destination_city[:3].upper())

from datetime import datetime, timedelta
import re

def normalize_date(value, default_date):
    # If value is already a valid date string
    try:
        datetime.strptime(value, '%Y-%m-%d')
        return value
    except:
        pass
    # If value is a month name like "July"
    try:
        if re.match(r'^[A-Za-z]+$', value.strip()):
            month_number = datetime.strptime(value.strip(), "%B").month
            today = datetime.today()
            year = today.year if today.month <= month_number else today.year + 1
            date_obj = datetime(year, month_number, 10)  # Use 10th as default day
            return date_obj.strftime('%Y-%m-%d')
    except:
        pass
    # Fallback to default
    return default_date

# Default dates
default_departure = (datetime.now() + timedelta(days=30)).strftime('%Y-%m-%d')
default_return = (datetime.now() + timedelta(days=37)).strftime('%Y-%m-%d')

# Get from criteria or default
raw_departure = criteria.get('departure_date', default_departure)
raw_return = criteria.get('return_date', default_return)

# Normalize dates
departure_date = normalize_date(raw_departure, default_departure)
return_date = normalize_date(raw_return, default_return)

# Passenger info
adults = criteria.get('adults', 2)
children = criteria.get('children', 1)

# Class
travel_class = criteria.get('travel_class', 'ECONOMY')

# Call flight API
print(f"Searching for flights from {origin} to {destination} on {departure_date} returning {return_date}...")
flight_offers = flight_api.search_flights(
    origin_location_code=origin,
    destination_location_code=destination,
    departure_date=departure_date,
    return_date=return_date,
    adults=adults,
    children=children,
    travel_class=travel_class,
    currency_code="USD",
    max_results=5
)

# Output
print(f"Found {len(flight_offers)} flight offers")
if flight_offers:
    flights_df = data_processor.parse_flight_data(flight_offers)
    flights_df.head()



Searching for flights from NYC to PAR on 2025-07-10 returning 2025-07-10...
Found 5 flight offers


## 5. Search for Hotels

Now, let's search for hotels at the destination:

In [10]:
# Search for hotels
print(f"Searching for hotels in {destination} from {departure_date} to {return_date}...")
hotel_offers = hotel_api.search_hotels(
    city_code=destination,
    check_in_date=departure_date,
    check_out_date=return_date,
    adults=adults,
    radius=5,
    radius_unit="KM",
    currency="USD",
    max_results=5
)

# Display hotel offers
print(f"Found {len(hotel_offers)} hotel offers")
if hotel_offers:
    # Parse hotel data into a DataFrame
    hotels_df = data_processor.parse_hotel_data(hotel_offers)
    hotels_df.head()

Searching for hotels in PAR from 2025-07-10 to 2025-07-10...


ERROR:hotel_api:Error searching for hotels: object of type 'NoneType' has no len()


Found 0 hotel offers


In [11]:
# Search for hotels
print(f"Searching for hotels in {destination} from {departure_date} to {return_date}...")
hotel_offers = hotel_api.search_hotels(
    city_code=destination,
    check_in_date=departure_date,
    check_out_date=return_date,
    adults=adults,
    radius=5,
    radius_unit="KM",
    currency="USD",
    max_results=5
)

# Validate response before accessing it
if hotel_offers is None:
    logger.error("Hotel API returned None. No hotel offers found.")
    hotel_offers = []
else:
    print(f"Found {len(hotel_offers)} hotel offers")

# Display hotel offers
if hotel_offers:
    hotels_df = data_processor.parse_hotel_data(hotel_offers)
    hotels_df.head()
else:
    print("No hotel offers to display.")


Searching for hotels in PAR from 2025-07-10 to 2025-07-10...


ERROR:hotel_api:Error searching for hotels: object of type 'NoneType' has no len()


Found 0 hotel offers
No hotel offers to display.


## 6. Create Travel Packages

Now, let's combine flights and hotels to create travel packages:

In [13]:
from datetime import datetime, timedelta

# Parse dates
departure_date_obj = datetime.strptime(departure_date, "%Y-%m-%d")
return_date_obj = datetime.strptime(return_date, "%Y-%m-%d")

# If dates are the same or invalid, adjust return date
if return_date_obj <= departure_date_obj:
    return_date_obj = departure_date_obj + timedelta(days=1)
    return_date = return_date_obj.strftime("%Y-%m-%d")


In [15]:
try:
    print(f"Searching for hotels in {destination} from {departure_date} to {return_date}...")
    hotel_offers = hotel_api.search_hotels(
        city_code=destination,
        check_in_date=departure_date,
        check_out_date=return_date,
        adults=adults,
        radius=5,
        radius_unit="KM",
        currency="USD",
        max_results=5
    )

    if hotel_offers is not None:
        print(f"Found {len(hotel_offers)} hotel offers")
        if hotel_offers:
            hotels_df = data_processor.parse_hotel_data(hotel_offers)
            hotels_df.head()
        else:
            print("No hotel offers to display.")
    else:
        print("Hotel API returned no results (None).")
except Exception as e:
    logger.error(f"Error searching for hotels: {e}")


Searching for hotels in PAR from 2025-07-10 to 2025-07-11...


ERROR:hotel_api:Error searching for hotels: object of type 'NoneType' has no len()


Found 0 hotel offers
No hotel offers to display.


In [14]:
# Create travel packages
print("Creating travel packages...")
packages = package_api.create_single_destination_package(
    origin_location_code=origin,
    destination_location_code=destination,
    departure_date=departure_date,
    return_date=return_date,
    adults=adults,
    children=children,
    travel_class=travel_class,
    currency_code="USD",
    max_flight_results=5,
    max_hotel_results=5
)

# Display packages
print(f"Created {len(packages)} travel packages")
if packages:
    # Parse package data into a DataFrame
    packages_df = data_processor.parse_package_data(packages)
    packages_df.head()

Creating travel packages...


ERROR:hotel_api:Error searching for hotels: object of type 'NoneType' has no len()


Created 0 travel packages


## 7. Get AI-Powered Recommendations

Now, let's use the travel agent to generate personalized recommendations based on the available packages:

In [ ]:
# Generate recommendations
if packages:
    print("Generating recommendations...")
    recommendations = travel_agent.generate_travel_recommendations(packages[:3], criteria)
    print("\nRecommendations:")
    print(recommendations)

## 8. View Package Details

Let's view the details of a specific package:

In [ ]:
# View package details
if packages:
    package_id = 0  # View the first package
    print(f"Viewing details for package {package_id}...")
    package_details = travel_agent.format_package_details(packages[package_id], detailed=True)
    print("\nPackage Details:")
    print(package_details)

## 9. Ask Travel-Related Questions

Finally, let's ask the travel agent some questions:

In [ ]:
# Ask a question
question = "What's the best time to visit Paris?"
print(f"Question: {question}")

# Prepare context with current search results and criteria
context = {
    'current_criteria': criteria,
    'has_search_results': len(packages) > 0,
    'number_of_packages': len(packages) if packages else 0,
    'price_range': {
        'min': min([p['total_price'] for p in packages]) if packages else None,
        'max': max([p['total_price'] for p in packages]) if packages else None,
        'currency': packages[0]['currency'] if packages else None
    }
}

# Get answer from agent
answer = travel_agent.answer_travel_question(question, context)
print("\nAnswer:")
print(answer)

Question: What's the best time to visit Paris?

Answer:
The best time to visit Paris is during the spring (April-May) and autumn (September-October), when the weather is mild and pleasant, with average temperatures ranging from 15°C to 25°C (59°F to 77°F). These periods offer a great balance of comfortable weather and smaller crowds, making it ideal for exploring the city's famous landmarks and attractions.


## 10. Launch Gradio Web Interface

If you want to use the web interface instead of the notebook, you can launch the Gradio app:

In [13]:
from frontend.app import app as gradio_app

# Launch the app with sharing enabled
gradio_app.launch(share=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1ef62858c470233fdd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
error
Connection errored out.

In [ ]:
"""
Hotel API Module - Handles interactions with hotel booking APIs
"""
import logging
from datetime import datetime
from amadeus import Client, ResponseError
from typing import Dict, List, Optional, Any, Union

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('hotel_api')

class HotelAPI:
    """
    Hotel API class to handle interactions with Amadeus hotel booking API
    """
    def __init__(self, api_key: str, api_secret: str):
        """
        Initialize HotelAPI with Amadeus credentials

        Args:
            api_key: Amadeus API key
            api_secret: Amadeus API secret
        """
        self.api_key = api_key
        self.api_secret = api_secret
        self.client = Client(
            client_id=api_key,
            client_secret=api_secret
        )
        logger.info("HotelAPI initialized with Amadeus client")

    # def search_hotels(self,
    #                  city_code: str,
    #                  check_in_date: str,
    #                  check_out_date: str,
    #                  adults: int = 1,
    #                  radius: int = 5,
    #                  radius_unit: str = "KM",
    #                  hotel_name: Optional[str] = None,
    #                  chains: Optional[List[str]] = None,
    #                  ratings: Optional[List[str]] = None,
    #                  amenities: Optional[List[str]] = None,
    #                  price_range: Optional[str] = None,
    #                  currency: Optional[str] = None,
    #                  lang: str = "EN",
    #                  max_results: int = 10) -> List[Dict[str, Any]]:
    #     """
    #     Search for hotels using Amadeus API

    #     Args:
    #         city_code: IATA city code
    #         check_in_date: Check-in date in YYYY-MM-DD format
    #         check_out_date: Check-out date in YYYY-MM-DD format
    #         adults: Number of adults
    #         radius: Radius around the city center in kilometers
    #         radius_unit: Unit of radius (KM or MILE)
    #         hotel_name: Name of the hotel (optional)
    #         chains: List of hotel chain codes (optional)
    #         ratings: List of hotel ratings (optional)
    #         amenities: List of amenity codes (optional)
    #         price_range: Price range in format min-max (optional)
    #         currency: Currency code (optional)
    #         lang: Language code
    #         max_results: Maximum number of results to return

    #     Returns:
    #         List of hotel offers
    #     """
    #     # Set up parameters for the request
    #     params = {
    #         'cityCode': city_code,
    #         'checkInDate': check_in_date,
    #         'checkOutDate': check_out_date,
    #         'adults': adults,
    #         'radius': radius,
    #         'radiusUnit': radius_unit,
    #         'hotelName': hotel_name,
    #         'chainCodes': chains,
    #         'ratings': ratings,
    #         'amenities': amenities,
    #         'priceRange': price_range,
    #         'currency': currency,
    #         'lang': lang,
    #         'max': max_results
    #     }

    #     # Remove None values
    #     params = {k: v for k, v in params.items() if v is not None}
    #     logger.info(f"Searching hotels with parameters: {params}")

    #     try:
    #         response = self.client.shopping.hotel_offers.get(**params)
    #         logger.info(f"Found {len(response.data)} hotel offers")
    #         return response.data
    #     except ResponseError as error:
    #         logger.error(f"Error searching hotels: {error}")
    #         logger.error(f"Error code: {error.code}")
    #         logger.error(f"Error message: {error.description}")
    #         if hasattr(error, 'response'):
    #             logger.error(f"Full error response: {error.response.body}")
    #         return []
    # Assuming the HotelAPI class is in api/hotel_api.py
# Open the file and modify the search_hotels method as follows:

    def search_hotels(self, city_code, check_in_date, check_out_date, adults, radius, radius_unit,
                      hotel_name=None, chains=None, ratings=None, amenities=None, price_range=None,
                      currency="USD", lang="en-US", max_results=10):
        """
        Searches for hotel offers.

        Args:
            city_code (str): The IATA city code of the destination.
            check_in_date (str): The check-in date (YYYY-MM-DD).
            check_out_date (str): The check-out date (YYYY-MM-DD).
            adults (int): The number of adults.
            radius (int): The search radius.
            radius_unit (str): The unit of the radius (KM or MILE).
            hotel_name (str, optional): Search by hotel name. Defaults to None.
            chains (str, optional): Filter by hotel chains (comma-separated). Defaults to None.
            ratings (str, optional): Filter by star ratings (comma-separated). Defaults to None.
            amenities (str, optional): Filter by amenities (comma-separated). Defaults to None.
            price_range (str, optional): Filter by price range (e.g., '100-500'). Defaults to None.
            currency (str, optional): The currency for prices. Defaults to "USD".
            lang (str, optional): The language for results. Defaults to "en-US".
            max_results (int, optional): The maximum number of results to return. Defaults to 10.

        Returns:
            list: A list of hotel offers.
        """
        params = {
            'cityCode': city_code,
            'checkInDate': check_in_date,
            'checkOutDate': check_out_date,
            'adults': adults,
            'radius': radius,
            'radiusUnit': radius_unit,
            'currency': currency,
            'lang': lang,
            'pageSize': max_results  # Use pageSize for max results
        }

        if hotel_name:
            params['hotelName'] = hotel_name
        if chains:
            params['chains'] = chains
        if ratings:
            params['ratings'] = ratings
        if amenities:
            params['amenities'] = amenities
        if price_range:
            params['priceRange'] = price_range

        try:
            # Modify this line to directly use the client's get method
            response = self.client.get('/v1/shopping/hotel-offers', **params)
            logger.info(f"Found {len(response.data)} hotel offers")
            return response.data
        except Exception as e:
            logger.error(f"Error searching for hotels: {e}")
            return []
    def search_hotel_by_geocode(self,
                              latitude: float,
                              longitude: float,
                              check_in_date: str,
                              check_out_date: str,
                              adults: int = 1,
                              radius: int = 5,
                              radius_unit: str = "KM",
                              hotel_name: Optional[str] = None,
                              chains: Optional[List[str]] = None,
                              ratings: Optional[List[str]] = None,
                              amenities: Optional[List[str]] = None,
                              price_range: Optional[str] = None,
                              currency: Optional[str] = None,
                              lang: str = "EN",
                              max_results: int = 10) -> List[Dict[str, Any]]:
        """
        Search for hotels using geocode (latitude and longitude) using Amadeus API

        Args:
            latitude: Latitude of the location
            longitude: Longitude of the location
            check_in_date: Check-in date in YYYY-MM-DD format
            check_out_date: Check-out date in YYYY-MM-DD format
            adults: Number of adults
            radius: Radius around the location in kilometers
            radius_unit: Unit of radius (KM or MILE)
            hotel_name: Name of the hotel (optional)
            chains: List of hotel chain codes (optional)
            ratings: List of hotel ratings (optional)
            amenities: List of amenity codes (optional)
            price_range: Price range in format min-max (optional)
            currency: Currency code (optional)
            lang: Language code
            max_results: Maximum number of results to return

        Returns:
            List of hotel offers
        """
        # Set up parameters for the request
        params = {
            'latitude': latitude,
            'longitude': longitude,
            'checkInDate': check_in_date,
            'checkOutDate': check_out_date,
            'adults': adults,
            'radius': radius,
            'radiusUnit': radius_unit,
            'hotelName': hotel_name,
            'chainCodes': chains,
            'ratings': ratings,
            'amenities': amenities,
            'priceRange': price_range,
            'currency': currency,
            'lang': lang,
            'max': max_results
        }

        # Remove None values
        params = {k: v for k, v in params.items() if v is not None}
        logger.info(f"Searching hotels by geocode with parameters: {params}")

        try:
            response = self.client.shopping.hotel_offers_by_hotel.get(**params)
            logger.info(f"Found {len(response.data)} hotel offers")
            return response.data
        except ResponseError as error:
            logger.error(f"Error searching hotels by geocode: {error}")
            logger.error(f"Error code: {error.code}")
            logger.error(f"Error message: {error.description}")
            if hasattr(error, 'response'):
                logger.error(f"Full error response: {error.response.body}")
            return []

    def get_hotel_offer(self, offer_id: str) -> Dict[str, Any]:
        """
        Get details of a specific hotel offer

        Args:
            offer_id: ID of the hotel offer

        Returns:
            Hotel offer details
        """
        try:
            response = self.client.shopping.hotel_offer(offer_id).get()
            logger.info(f"Successfully retrieved hotel offer with ID {offer_id}")
            return response.data
        except ResponseError as error:
            logger.error(f"Error getting hotel offer: {error}")
            logger.error(f"Error code: {error.code}")
            logger.error(f"Error message: {error.description}")
            if hasattr(error, 'response'):
                logger.error(f"Full error response: {error.response.body}")
            return {}
